In [1]:
import pandas as pd
import boto3
from botocore.exceptions import ClientError
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
endpoint_ending_mgc = '.magaluobjects.com/'
endpoint_url = 'https://br-se1.magaluobjects.com/'
profile_name = 'br-se1'

session = boto3.Session(profile_name=profile_name)
client = session.client('s3', endpoint_url= endpoint_url) 

objs = client.list_objects_v2(Bucket = 's3-specs-tests-reports-manual-parquet')
o = [o['Key'] for o in objs['Contents']]

client.download_file(Bucket = 's3-specs-tests-reports-manual-parquet', Key='execution_entity.parquet', Filename='execution_entity.parquet')
a = pd.read_parquet('execution_entity.parquet')

In [3]:
import os
oa =os.listdir('output')
a = [o for o in oa if 'parquet' in o]
a

['artifact.parquet',
 'execution_time.parquet',
 'tests.parquet',
 'failures.parquet',
 'execution_entity.parquet']

In [24]:
p = pd.read_parquet('./output/execution_time.parquet')
#for d in a:
    #p.append(pd.read_parquet('./output/'+ d))
f = p.drop(columns='Execution_Datetime').head(2)
f.columns

Index(['Execution_name', 'Execution_type', 'Number_Runs', 'Avg_Time',
       'Min_Time', 'Total_Time'],
      dtype='object')

In [35]:
# Melt the DataFrame
melted_df = pd.melt(
    f,
    id_vars=['Execution_name', 'Execution_type', 'Number_Runs'],  # Columns to keep as-is
    value_vars=['Avg_Time', 'Min_Time', 'Total_Time'],  # Columns to merge
    var_name='type_time',  # New column name for the time types
    value_name='time_values'  # New column name for the time values
).reset_index(drop=True)

print("Transformed DataFrame:")
display(melted_df)

Transformed DataFrame:


,Execution_name,Execution_type,Number_Runs,type_time,time_values
0,test_boto_multipart_upload_with_cold_storage_c...,teardown duration,1,Avg_Time,3.675
1,test_boto_upload_object_with_cold_storage_class,teardown duration,1,Avg_Time,1.436
2,test_boto_multipart_upload_with_cold_storage_c...,teardown duration,1,Min_Time,3.675
3,test_boto_upload_object_with_cold_storage_class,teardown duration,1,Min_Time,1.436
4,test_boto_multipart_upload_with_cold_storage_c...,teardown duration,1,Total_Time,3.675
5,test_boto_upload_object_with_cold_storage_class,teardown duration,1,Total_Time,1.436


In [34]:
melted_df.to_dict('records')

[{'Execution_name': 'test_boto_multipart_upload_with_cold_storage_class',
  'Execution_type': ' teardown duration ',
  'Number_Runs': '1',
  'type_time': 'Avg_Time',
  'time_values': 3.675},
 {'Execution_name': 'test_boto_upload_object_with_cold_storage_class',
  'Execution_type': ' teardown duration ',
  'Number_Runs': '1',
  'type_time': 'Avg_Time',
  'time_values': 1.436},
 {'Execution_name': 'test_boto_multipart_upload_with_cold_storage_class',
  'Execution_type': ' teardown duration ',
  'Number_Runs': '1',
  'type_time': 'Min_Time',
  'time_values': 3.675},
 {'Execution_name': 'test_boto_upload_object_with_cold_storage_class',
  'Execution_type': ' teardown duration ',
  'Number_Runs': '1',
  'type_time': 'Min_Time',
  'time_values': 1.436},
 {'Execution_name': 'test_boto_multipart_upload_with_cold_storage_class',
  'Execution_type': ' teardown duration ',
  'Number_Runs': '1',
  'type_time': 'Total_Time',
  'time_values': 3.675},
 {'Execution_name': 'test_boto_upload_object_with